# 1. Introducción

### Train: es para entrenar el algoritmo
### Test: es para subir las predicciones

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor


from utilidades.validar import validar

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

dfinicial = pd.read_csv("data/train_limpio.csv")
dftest = pd.read_csv("data/test_limpio.csv")

xg = pd.read_csv('submits/submit_xg_prueba.csv')

In [2]:
pd.set_option('display.max_columns', 23) #ver todas las columnas
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

### El ejemplo de respuesta parece tener una fila como índice

# 4. Algoritmos Probados

In [3]:
def generar_subida(DFINICIAL, metodo, DFTESTEAR):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
    X, y = DFINICIAL.drop( columns = ['precio'] ), DFINICIAL["precio"]

    reg = metodo()
    reg.fit(X,y)
    
    subida = pd.DataFrame( reg.predict(DFTESTEAR) , columns=["target"])
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    # Heurística: si vale menos de 310 000, ponerle 310 000
    subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    
    # Heurística 2: si vale más de 12 500 000, ponerle 12 500 000
    subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    subida.to_csv(f"submits/prueba_esteban.csv", index = False)
    
    validar(X, y, metodo, False)
    
    return subida

### 4.1. Linear Regression

In [4]:
dfinicial1 = dfinicial
dftest1 = dftest

#dfinicial1 = dfinicial.drop( columns = [ 'anio', 'metrostotales'
#                           ])                                        
 
#dftest1 = dftest.drop( columns = [  'anio', 'metrostotales'
#                           ])   

subida = generar_subida( dfinicial1 ,  LinearRegression, dftest1)

Error de fold N°0 es : 934977.3264834604
Error de fold N°1 es : 930901.7623617502
Error de fold N°2 es : 931381.001369224
Error de fold N°3 es : 928749.4780054106
Error de fold N°4 es : 923816.5556214594


### 4.2. KNN

In [5]:
def generar_subida2(DFINICIAL, metodo, DFTESTEAR, metrica1, metrica2):
    """
    X: datos de entrenamiento
    y: target de X
    metodo: algoritmo a utilizar
    """
    
    #ML
    X, y = DFINICIAL.drop( columns = ['precio'] ), dfinicial["precio"]
    reg = metodo(metric = metrica1, p = metrica2)
    reg.fit(X,y) #entrena como ajustarte a y (precios)
    subida = pd.DataFrame( reg.predict(DFTESTEAR) , columns=["target"]) #crea df con predicciones

    #Formateo
    subida.insert(0, "id", test["id"]) #test todavía tiene los ids
    subida['id'] = subida['id'].astype(np.uint32)
    subida['target'] = subida['target'].astype(np.uint32)
    
    #Heurísticas
    # si vale menos de 310 000, ponerle 310 000
    subida.loc[ subida['target'] <= 310000, 'target'] = 310000
    # si vale más de 12 500 000, ponerle 12 500 000
    subida.loc[ subida['target'] >= 12500000, 'target'] = 12500000
    
    subida.to_csv(f"submits/prueba_esteban.csv", index = False)
    
    validar(X, y, metodo) # corregir
    
    return subida

In [6]:
dfinicial1 = dfinicial
dftest1 = dftest

subida = generar_subida2( dfinicial1 ,  KNeighborsRegressor, dftest1,'euclidean', 2)

Error de fold N°0 es : 1227347.693124284
Error de fold N°1 es : 1219512.1536361137
Error de fold N°2 es : 1225856.3189577935
Error de fold N°3 es : 1219610.6076856868
Error de fold N°4 es : 1218826.79360932


In [7]:
'''dfinicial1 = dfinicial
dftest1 = dftest

subida = generar_subida2( dfinicial1 ,  KNeighborsRegressor, dftest1,'cosine', 2)
'''

"dfinicial1 = dfinicial\ndftest1 = dftest\n\nsubida = generar_subida2( dfinicial1 ,  KNeighborsRegressor, dftest1,'cosine', 2)\n"

In [8]:
dfinicial1 = dfinicial
dftest1 = dftest

subida = generar_subida2( dfinicial1 ,  KNeighborsRegressor, dftest1,'minkowski', 1 ) #Manhattan

Error de fold N°0 es : 1227347.693124284
Error de fold N°1 es : 1219512.1536361137
Error de fold N°2 es : 1225856.3189577935
Error de fold N°3 es : 1219610.6076856868
Error de fold N°4 es : 1218826.79360932
